In [28]:
import os, pandas, numpy
from sklearn.model_selection import KFold
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics

In [20]:

df1=pandas.read_csv("PS_20174392719_1491204439457_log.csv", na_values=['NA', '?'])
pandas.set_option('display.max_columns', None)

df1

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.00,160296.36,M1979787155,0.00,0.00,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.00,19384.72,M2044282225,0.00,0.00,0,0
2,1,TRANSFER,181.00,C1305486145,181.00,0.00,C553264065,0.00,0.00,1,0
3,1,CASH_OUT,181.00,C840083671,181.00,0.00,C38997010,21182.00,0.00,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.00,29885.86,M1230701703,0.00,0.00,0,0
...,...,...,...,...,...,...,...,...,...,...,...
6362615,743,CASH_OUT,339682.13,C786484425,339682.13,0.00,C776919290,0.00,339682.13,1,0
6362616,743,TRANSFER,6311409.28,C1529008245,6311409.28,0.00,C1881841831,0.00,0.00,1,0
6362617,743,CASH_OUT,6311409.28,C1162922333,6311409.28,0.00,C1365125890,68488.84,6379898.11,1,0
6362618,743,TRANSFER,850002.52,C1685995037,850002.52,0.00,C2080388513,0.00,0.00,1,0


In [21]:
df1=df1.reindex(numpy.random.permutation(df1.index))
mask=numpy.random.rand(len(df1))<0.8
trainDF=pandas.DataFrame(df1[mask])
validationDF=pandas.DataFrame(df1[~mask])

print(f"Training DF: {len(trainDF)}")
print(f"Validation DF: {len(validationDF)}")

Training DF: 5089921
Validation DF: 1272699


In [22]:
df2=pandas.read_csv("PS_20174392719_1491204439457_log.csv", na_values=['NA', '?'])
df2=df2.reindex(numpy.random.permutation(df2.index))
kf =KFold(5)

fold = 1
for train_index, validate_index in kf.split(df2):
    trainDF=pandas.DataFrame(df2.iloc[train_index, :])
    validateDF=pandas.DataFrame(df2.iloc[validate_index])
    print(f"Fold#{fold}, Training Size: {len(trainDF)},  Validation Size: {len(validationDF)}")
    fold+=1

Fold#1, Training Size: 5090096,  Validation Size: 1272699
Fold#2, Training Size: 5090096,  Validation Size: 1272699
Fold#3, Training Size: 5090096,  Validation Size: 1272699
Fold#4, Training Size: 5090096,  Validation Size: 1272699
Fold#5, Training Size: 5090096,  Validation Size: 1272699


In [29]:
#loading the dataset as dataframe
df3=pandas.read_csv("PS_20174392719_1491204439457_log.csv", na_values=['NA', '?'])

#shuffling the data
df3=df3.reindex(numpy.random.permutation(df3.index))

enc=LabelEncoder()
column_name= "type"
space=1
enc.fit(df3[column_name])
enc_type = enc.transform(df3[column_name])
df3.insert(space,"encoded_" + column_name,enc_type)
df3.pop("type")

column_name= "nameOrig"
space=3
enc.fit(df3[column_name])
enc_type = enc.transform(df3[column_name])
df3.insert(space,"encoded_" + column_name,enc_type)
df3.pop("nameOrig")

column_name= "nameDest"
space=6
enc.fit(df3[column_name])
enc_type = enc.transform(df3[column_name])
df3.insert(space,"encoded_" + column_name,enc_type)
df3.pop("nameDest")


result = []
for x in df3.columns:
    if x != 'isFraud':
        result.append(x)

# x and y data
X=df3[result].values
y=df3['isFraud'].values

#Perceptron creation
ppn=Perceptron(max_iter=70, tol=0.0001, eta0=1)

#k-fold splitting to 5(for now, might be good with 10 as well)
kf =KFold(5)

fold = 1
for train_index, validate_index in kf.split(X, y):
    ppn.fit(X[train_index], y[train_index])
    y_test=y[validate_index]
    y_pred=ppn.predict(X[validate_index])
    
    
    print(f"Fold#{fold}, Training Size: {len(X[train_index])},  Validation Size: {len(X[validate_index])}")
    print("Accuracy: %.2f" % accuracy_score(y_test, y_pred))
    
    #precision check
    precision = metrics.average_precision_score(y_test, y_pred)
    print("precision score: {}".format(precision))
    
    #validateDF=pandas.DataFrame(df3.iloc[validate_index])
    #print(f"Fold#{fold}, Training Size: {len(trainDF)},  Validation Size: {len(validationDF)}")
    fold+=1

Fold#1, Training Size: 5090096,  Validation Size: 1272524
Accuracy: 1.00
precision score: 0.2464982105759115
Fold#2, Training Size: 5090096,  Validation Size: 1272524
Accuracy: 1.00
precision score: 0.27307844990168295
Fold#3, Training Size: 5090096,  Validation Size: 1272524
Accuracy: 1.00
precision score: 0.36164210983013245
Fold#4, Training Size: 5090096,  Validation Size: 1272524
Accuracy: 1.00
precision score: 0.26231230534879313
Fold#5, Training Size: 5090096,  Validation Size: 1272524
Accuracy: 1.00
precision score: 0.2466430979614848


In [24]:
df3

,step,encoded_type,amount,encoded_nameOrig,oldbalanceOrg,newbalanceOrig,encoded_nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
2770695,213,4,2379304.82,693111,0.00,0.00,7539,4595338.66,6974643.49,0,0
1151650,131,3,5684.06,784812,0.00,0.00,1047866,0.00,0.00,0,0
5921171,404,3,4022.46,2150736,0.00,0.00,2677404,0.00,0.00,0,0
1795436,162,3,16407.69,2084164,99165.29,82757.61,2045484,0.00,0.00,0,0
5584905,394,1,433792.61,3071561,0.00,0.00,564325,1594889.12,2028681.73,0,0
...,...,...,...,...,...,...,...,...,...,...,...
6330594,690,1,221477.03,1175754,7263.00,0.00,139574,4950772.86,5172249.90,0,0
3534773,259,1,7111.96,4735043,0.00,0.00,297410,87005.63,94117.59,0,0
2129032,183,0,205870.59,6163909,4055120.31,4260990.90,75609,384034.19,178163.60,0,0
4562987,327,3,18230.74,933115,102530.15,84299.41,1707909,0.00,0.00,0,0
